In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder.
         appName("working with tabular data").
         getOrCreate())

In [3]:
import os
import pyspark.sql.functions as F

In [4]:
DIRECTORY = r"C:\Users\oluwa\Documents\broadcast_logs"

In [5]:
#%%bash

#!ls DIRECTORY
files = os.listdir(DIRECTORY)

for f in files:
    print(f)
print("-"*100)
for f in files:
    print(os.path.join(DIRECTORY, f))

BroadcastLogs_2018_Q3_M8.CSV
BroadcastLogs_2018_Q3_M8_sample.CSV
data dictionary.doc
ReferenceTables
----------------------------------------------------------------------------------------------------
C:\Users\oluwa\Documents\broadcast_logs\BroadcastLogs_2018_Q3_M8.CSV
C:\Users\oluwa\Documents\broadcast_logs\BroadcastLogs_2018_Q3_M8_sample.CSV
C:\Users\oluwa\Documents\broadcast_logs\data dictionary.doc
C:\Users\oluwa\Documents\broadcast_logs\ReferenceTables


In [6]:
logs = spark.read.csv(
    os.path.join(DIRECTORY, "BroadcastLogs_2018_Q3_M8.csv"),
    sep = "|",
    header = True,
    inferSchema = True,
    timestampFormat = "yyyy-MM-dd",
)

In [ ]:
logs.printSchema()

In [ ]:
logs.select("BroadcastLogID", "LogServiceID", "LogDate").show(5, False)

In [ ]:
logs = logs.select(*[x for x in logs.columns if x not in ["BroadcastLogID", "LogServiceID", "LogDate"]])

In [ ]:
import pandas as pd

"Language1" in logs.columns

logs.columns

In [7]:
logs.select('BroadcastLogID',
 'LogServiceID',
 'LogDate').show(5, truncate=False)

+--------------+------------+-------------------+
|BroadcastLogID|LogServiceID|LogDate            |
+--------------+------------+-------------------+
|1196192316    |3157        |2018-08-01 00:00:00|
|1196192317    |3157        |2018-08-01 00:00:00|
|1196192318    |3157        |2018-08-01 00:00:00|
|1196192319    |3157        |2018-08-01 00:00:00|
|1196192320    |3157        |2018-08-01 00:00:00|
+--------------+------------+-------------------+
only showing top 5 rows



In [8]:
#exclude 'BroadcastLogID' and 'SequenceNO' columns

logs_copy = logs.select(*[x for x in logs.columns if x not in ['BroadcastLogID', 'SequenceNO', 'LogServiceID']])

In [ ]:
logs_copy.columns

In [9]:
logs_copy.select(F.col('LogDate')).show(5, False)

+-------------------+
|LogDate            |
+-------------------+
|2018-08-01 00:00:00|
|2018-08-01 00:00:00|
|2018-08-01 00:00:00|
|2018-08-01 00:00:00|
|2018-08-01 00:00:00|
+-------------------+
only showing top 5 rows



In [10]:
logs_copy.select(F.substring(F.col('LogDate'), 1, 4)).distinct().show(5)

+------------------------+
|substring(LogDate, 1, 4)|
+------------------------+
|                    2018|
+------------------------+



In [ ]:
logs_copy.select(
    F.col('Duration').alias("full_duration")).show(5, False)

In [11]:
logs_copy.select(
    F.col('Duration').alias('full'),
    F.col('Duration').substr(1,2).cast('int').alias('hour'),
    F.col('Duration').substr(4,2).cast('int').alias('min'),
    F.col('Duration').substr(7,2).cast('int').alias('sec'),
    (F.col('Duration').substr(1,2).cast('int') * 60 * 60 +
    F.col('Duration').substr(4,2).cast('int') * 60 +
    F.col('Duration').substr(7,2).cast('int')).alias('duration_secs')
).distinct().show(5, False)

+----------------+----+---+---+-------------+
|full            |hour|min|sec|duration_secs|
+----------------+----+---+---+-------------+
|00:59:52.0000000|0   |59 |52 |3592         |
|00:16:31.0000000|0   |16 |31 |991          |
|00:13:30.0000000|0   |13 |30 |810          |
|00:48:17.0000000|0   |48 |17 |2897         |
|00:00:19.0000000|0   |0  |19 |19           |
+----------------+----+---+---+-------------+
only showing top 5 rows



In [12]:
logs_copy = logs.withColumn('duration_secs',
F.col('Duration').substr(1,2).cast('int') * 60 * 60 +
F.col('Duration').substr(4,2).cast('int') * 60 +
F.col('Duration').substr(7,2).cast('int'))

In [ ]:
logs_copy.printSchema()

In [ ]:
logs_copy.select(F.col('Duration'), F.col('duration_secs')).show(5, False)

In [ ]:
logs_copy = logs_copy.select(*[x.lower() for x in logs_copy.columns])

In [ ]:
logs_copy.printSchema()

In [ ]:
logs_copy.columns

In [13]:
for w in logs_copy.columns:
    logs_copy.describe(w).show()

+-------+--------------------+
|summary|      BroadcastLogID|
+-------+--------------------+
|  count|             7169318|
|   mean|1.2168893983502293E9|
| stddev|1.4982198462892085E7|
|    min|          1195788151|
|    max|          1249453843|
+-------+--------------------+

+-------+------------------+
|summary|      LogServiceID|
+-------+------------------+
|  count|           7169318|
|   mean|3453.8804215407936|
| stddev|200.44137201583823|
|    min|              3157|
|    max|              3925|
+-------+------------------+

+-------+
|summary|
+-------+
|  count|
|   mean|
| stddev|
|    min|
|    max|
+-------+

+-------+------------------+
|summary|        SequenceNO|
+-------+------------------+
|  count|           7169318|
|   mean|13976.638132943748|
| stddev|  9330.05576809833|
|    min|                 1|
|    max|             41126|
+-------+------------------+

+-------+-------------------+
|summary|AudienceTargetAgeID|
+-------+-------------------+
|  count|      

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "C:\PySpark\spark-3.3.2-bin-hadoop2\python\lib\py4j-0.10.9.5-src.zip\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\PySpark\spark-3.3.2-bin-hadoop2\python\lib\py4j-0.10.9.5-src.zip\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "c:\users\oluwa\appdata\local\programs\python\python39\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [17]:
logs_copy.select(F.col('LogDate')).show(5)

+-------------------+
|            LogDate|
+-------------------+
|2018-08-01 00:00:00|
|2018-08-01 00:00:00|
|2018-08-01 00:00:00|
|2018-08-01 00:00:00|
|2018-08-01 00:00:00|
+-------------------+
only showing top 5 rows



In [ ]:
logs_copy = logs.withColumn('duration_secs',
F.col('Duration').substr(1,2).cast('int') * 60 * 60 +
F.col('Duration').substr(4,2).cast('int') * 60 +
F.col('Duration').substr(7,2).cast('int'))

In [31]:
logs_copy = logs_copy.withColumn('seconds_duration',
F.col('Duration').substr(1,2).cast('int')*60*60 +
F.col('Duration').substr(4,2).cast('int')*60 +
F.col('duration').substr(7,2).cast('int'))

In [35]:
logs_copy.select(F.col('seconds_duration'), F.col('Duration').substr(1,8)).show(5, False)

+----------------+-------------------------+
|seconds_duration|substring(Duration, 1, 8)|
+----------------+-------------------------+
|7200            |02:00:00                 |
|30              |00:00:30                 |
|15              |00:00:15                 |
|15              |00:00:15                 |
|15              |00:00:15                 |
+----------------+-------------------------+
only showing top 5 rows

